In [1]:
# pandas
import pandas as pd 

In [2]:
import PySimpleGUI as sg
def selection(option):
    #sg.theme('DarkAmber')   # Add a touch of color
    #select the variable we want to calculate
    options = option

    # All the stuff inside your window.
    layout = [ 
            [sg.Text('Select one->'), sg.Listbox(options,select_mode=sg.LISTBOX_SELECT_MODE_SINGLE,size=(20,len(options)))],
            [sg.Button('Ok'), sg.Button('Cancel')]
        ]

    # Create the Window
    window = sg.Window('Make your choice', layout)

    # Event Loop to process "events" and get the "values" of the input
    while True:
        event, values = window.read()
        print( f"event={event}" )
        if event is None or event == 'Ok' or event == 'Cancel': # if user closes window or clicks cancel
            break
        
    # close  the window        
    window.close()

    if event == "Cancel":
        print( "You cancelled" )
    else:
        print('You entered ', values[0])
        sg.popup( f"You selected {values[0]}" )        
    return values


In [3]:
#nama_10_a64 National accounts aggregates by industry (up to NACE A*64)
#NAMA_10_A64_P5 Gross capital formation by industry (up to NACE A*64)
#National accounts employment data by industry (up to NACE A*64)

import eurostat
df = eurostat.get_data_df('env_ac_pefa04', flags=False)
df

,nace_r2,indic_pefa,unit,geo\time,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
0,A,EPRD_DOM,TJ,AT,NaN,129220.1,128165.0,140948.3,135707.7,135349.5,130721.5,145929.1,NaN,NaN,NaN,NaN,NaN
1,A,EPRD_DOM,TJ,BE,NaN,54319.3,56436.6,53044.7,56316.3,55854.1,49723.4,59804.4,58362.3,51543.2,62270.8,44484.1,37713.2
2,A,EPRD_DOM,TJ,BG,NaN,63534.7,59518.4,42071.3,42380.8,45997.0,40781.6,NaN,NaN,NaN,NaN,NaN,NaN
3,A,EPRD_DOM,TJ,CH,NaN,46497.3,44054.4,46334.0,45709.9,41672.8,40392.7,46510.8,42999.4,37945.3,42814.8,NaN,NaN
4,A,EPRD_DOM,TJ,CY,NaN,1075.0,1048.6,1038.8,916.2,823.5,565.8,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26828,U,WST_USE,TJ,RS,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26829,U,WST_USE,TJ,SE,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26830,U,WST_USE,TJ,SI,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26831,U,WST_USE,TJ,SK,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index([   'nace_r2', 'indic_pefa',       'unit',   'geo\time',         2020,
               2019,         2018,         2017,         2016,         2015,
               2014,         2013,         2012,         2011,         2010,
               2009,         2008],
      dtype='object')

In [5]:
#eliminate columns for years previous to 2011
years =[]
year_start=2008
year_study=2014
for y in range(year_start,year_study): 
    years.append(y)
years.append(2020)
df=df.drop(years, axis=1)#quitamos todas las columnas de añosque no nos interesan
#rename geo time column to be easier to handle because of the name
df.rename(columns={"geo\\time": "geo"}, inplace=True)
#We can get the unique values of the different variables

In [6]:
df.unit.unique()

array(['TJ'], dtype=object)

In [7]:
unit=['TJ']
df.indic_pefa.unique()

array(['EPRD_DOM', 'EPRD_HHCNS', 'EPRD_ICNS', 'NEI_EXT', 'NETDOM_EUSE',
       'NETDOM_EUSE_EP', 'NETDOM_EUSE_NEP', 'NRG_INP_OUT', 'WST_USE'],
      dtype=object)

[NEI_EXT] Extraction of natural energy inputs
[EPRD_DOM] Domestic production of energy products
[EPRD_ICNS] Intermediate consumption of energy products
[EPRD_HHCNS] Household consumption of energy products
[WST_USE] Use of waste for energetic purposes
[NETDOM_EUSE] Net domestic energy use
[NETDOM_EUSE_EP] Net domestic energy use for energy purposes
[NETDOM_EUSE_NEP] Net domestic energy use for non-energy purposes
[NRG_INP_OUT] Energy input and output 

In [8]:
cols = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI','FR', 'HR', 'HU', 'IE', 'IT', 'LT','LU', 'LV', 'MT','NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
EU27=['EU27_2020']
countries= set(cols)|set(EU27)
indic_pefa=['EPRD_DOM', 'EPRD_HHCNS', 'EPRD_ICNS', 'NEI_EXT', 'NETDOM_EUSE',
       'NETDOM_EUSE_EP', 'NETDOM_EUSE_NEP', 'NRG_INP_OUT', 'WST_USE']

In [9]:
df.nace_r2.unique()

array(['A', 'A01', 'A02', 'A03', 'B', 'C', 'C10-C12', 'C13-C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25',
       'C26', 'C27', 'C28', 'C29', 'C30', 'C31_C32', 'C33', 'CH_INV_PA',
       'D', 'E', 'E36', 'E37-E39', 'ENV', 'F', 'G', 'G45', 'G46', 'G47',
       'H', 'H49', 'H50', 'H51', 'H52', 'H53', 'HH', 'HH_HEAT', 'HH_OTH',
       'HH_TRA', 'I', 'J', 'J58', 'J59_J60', 'J61', 'J62_J63', 'K', 'K64',
       'K65', 'K66', 'L', 'L68A', 'M', 'M69_M70', 'M71', 'M72', 'M73',
       'M74_M75', 'N', 'N77', 'N78', 'N79', 'N80-N82', 'NRG_FLOW', 'O',
       'P', 'Q', 'Q86', 'Q87_Q88', 'R', 'R90-R92', 'R93', 'ROW_ACT', 'S',
       'S94', 'S95', 'S96', 'SD_SU', 'T', 'TOTAL', 'TOTAL_HH', 'U'],
      dtype=object)

In [10]:
weights_target = pd.read_excel("ecosystemweights.xlsx")
weights_target.NACE_R2.unique()

array(['A', 'C10-C12', 'C13-C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29',
       'C30', 'C31', 'C32', 'C33', 'D', 'E36', 'E37-E39', 'F', 'G45',
       'G46', 'G47', 'H49', 'H50', 'H51', 'H52', 'H53', 'I', 'J58',
       'J59_J60', 'J61', 'J62_J63', 'L', 'M69_M70', 'M71', 'M72', 'M73',
       'M74_M75', 'N77', 'N78', 'N79', 'N80', 'N81', 'N82', 'P', 'Q86',
       'Q87_Q88', 'R90-R92', 'R93', 'S94', 'S95', 'S96', 'T'],
      dtype=object)

In [11]:
nace_equal=set(weights_target.NACE_R2.unique())&set(df.nace_r2.unique())
nace_equal

{'A',
 'C10-C12',
 'C13-C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C33',
 'D',
 'E36',
 'E37-E39',
 'F',
 'G45',
 'G46',
 'G47',
 'H49',
 'H50',
 'H51',
 'H52',
 'H53',
 'I',
 'J58',
 'J59_J60',
 'J61',
 'J62_J63',
 'L',
 'M69_M70',
 'M71',
 'M72',
 'M73',
 'M74_M75',
 'N77',
 'N78',
 'N79',
 'P',
 'Q86',
 'Q87_Q88',
 'R90-R92',
 'R93',
 'S94',
 'S95',
 'S96',
 'T'}

In [12]:
nace_dif=set(weights_target.NACE_R2.unique())-set(df.nace_r2.unique())
nace_dif

{'C31', 'C32', 'N80', 'N81', 'N82'}

In [13]:
#ratios to divide N80_82 and 31_32. Value Added from SBS
ratios_values = pd.read_excel("ratioCN-Value added at factor cost - million euro.xlsx")
ratios_values.set_index('nace_r2', inplace=True)#nace como índice
ratios_values

,Value added at factor cost - million euro2011,Value added at factor cost - million euro2012,Value added at factor cost - million euro2013,Value added at factor cost - million euro2014,Value added at factor cost - million euro2015,Value added at factor cost - million euro2016,Value added at factor cost - million euro2017,Value added at factor cost - million euro2018,Value added at factor cost - million euro2019
nace_r2,,,,,,,,,
C31,0.398512,0.403322,0.400551,0.398158,0.400751,0.409175,0.388565,0.382716,0.390244
C32,0.601488,0.596678,0.599449,0.601842,0.599249,0.590825,0.611435,0.617284,0.609756
N80,0.165778,0.162820,0.158483,0.152327,0.157963,0.160093,0.155610,0.145835,0.145557
N81,0.462572,0.459438,0.466741,0.463845,0.467105,0.467350,0.469511,0.477816,0.490550
N82,0.371650,0.377742,0.374777,0.383828,0.374932,0.372558,0.374879,0.376349,0.363893


In [14]:
nace_include=['C31_C32','N80-N82']
nace_sel=set(nace_equal)|set(nace_include)
nace_sel

{'A',
 'C10-C12',
 'C13-C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31_C32',
 'C33',
 'D',
 'E36',
 'E37-E39',
 'F',
 'G45',
 'G46',
 'G47',
 'H49',
 'H50',
 'H51',
 'H52',
 'H53',
 'I',
 'J58',
 'J59_J60',
 'J61',
 'J62_J63',
 'L',
 'M69_M70',
 'M71',
 'M72',
 'M73',
 'M74_M75',
 'N77',
 'N78',
 'N79',
 'N80-N82',
 'P',
 'Q86',
 'Q87_Q88',
 'R90-R92',
 'R93',
 'S94',
 'S95',
 'S96',
 'T'}

In [15]:
#we select the rows that have the right unit, nace codes, items and countries
df_selection=df[df.indic_pefa.isin(indic_pefa)&df.nace_r2.isin(nace_sel)&df.geo.isin(countries)]
df_selection

,nace_r2,indic_pefa,unit,geo,2019,2018,2017,2016,2015,2014
0,A,EPRD_DOM,TJ,AT,129220.1,128165.0,140948.3,135707.7,135349.5,130721.5
1,A,EPRD_DOM,TJ,BE,54319.3,56436.6,53044.7,56316.3,55854.1,49723.4
2,A,EPRD_DOM,TJ,BG,63534.7,59518.4,42071.3,42380.8,45997.0,40781.6
4,A,EPRD_DOM,TJ,CY,1075.0,1048.6,1038.8,916.2,823.5,565.8
5,A,EPRD_DOM,TJ,CZ,153851.9,146487.9,142663.1,142775.1,130592.9,125663.4
...,...,...,...,...,...,...,...,...,...,...
25906,T,WST_USE,TJ,PT,0.0,0.0,0.0,0.0,0.0,0.0
25907,T,WST_USE,TJ,RO,0.0,0.0,0.0,0.0,0.0,0.0
25909,T,WST_USE,TJ,SE,0.0,0.0,0.0,0.0,0.0,0.0
25910,T,WST_USE,TJ,SI,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
import numpy as np


for z in range(0,len(indic_pefa)):
    #create a dataframe for the results
    results_compiled = pd.DataFrame()
    #print the variable that we are going to calculate
    print(indic_pefa[z])
    var=[indic_pefa[z]]#to pass it to a list to be able to apply isin
    #get the name of the variable and name it as the variable of interest
    variable_of_interest=indic_pefa[z]
    df_selection=df[df.indic_pefa.isin(var)&df.nace_r2.isin(nace_sel)&df.geo.isin(countries)]
    #loop to calculate values for years
    for j in range (2014,2020):
        year=j
        #we transpose the values for the year selected
        for i in range (year,year+1):
            df_target = pd.pivot_table(df_selection, values=i, index=['nace_r2'],columns=["geo"], aggfunc=np.sum, fill_value=0)#reorde
        df_target['sum'] = df_target[cols].sum(axis=1)#sums the columns of the countries to "sum" 
        df_target=df_target.drop(cols, axis=1)#eliminamos las columnas sumadas de países
        df_target.loc[df_target['EU27_2020']==0,'EU27_2020'] = df_target['sum']#now we can locate columns with 0 and make the substitution with the column "sum"
        moddf_target=df_target.drop('sum', axis=1)#we create a new table 
        #add rows C31,... and assign value with the ratio multiplying by the value of position C31_32
        year_ratio=j-2011 
        moddf_target.loc['C31'] = ratios_values.iloc[0][year_ratio]*moddf_target.loc['C31_C32','EU27_2020']
        moddf_target.loc['C32'] = ratios_values.iloc[1][year_ratio]*moddf_target.loc['C31_C32','EU27_2020']
        moddf_target.loc['N80'] = ratios_values.iloc[2][year_ratio]*moddf_target.loc['N80-N82','EU27_2020']
        moddf_target.loc['N81'] = ratios_values.iloc[3][year_ratio]*moddf_target.loc['N80-N82','EU27_2020']
        moddf_target.loc['N82'] = ratios_values.iloc[4][year_ratio]*moddf_target.loc['N80-N82','EU27_2020']
        #we eliminate rows C31_C32
        moddf_target = moddf_target.drop(['C31_C32', 'N80-N82'])
        pd.set_option('display.float_format', lambda x: '%.4f' % x)
        #cargamos los pesos en el dataframe weight  
        weights_target = pd.read_excel("ecosystemweights.xlsx")
        weights_target.set_index('NACE_R2', inplace=True)#nace como índice
        column_name=str(variable_of_interest)+str(year)
        weights_target[variable_of_interest]=weights_target.index.map(moddf_target['EU27_2020'])#busca en df_target el nace y copia el valor de EU27_2020 en la fila del nace the weights_target
        #creamos la matriz de resultados
        results=weights_target
        for i in range(0,14):
            results.iloc[:, i]=weights_target.iloc[:, i]*weights_target[variable_of_interest]#we multiply the total value added by the weight for each nace
        results.loc['total'] = results.select_dtypes(pd.np.number).sum() #we add the values of each colum in a row call total
        results_compiled[column_name]=results.loc['total']
    results_compiled = results_compiled.drop(variable_of_interest)
    #sum of the totals
    results_SMEs = results_compiled.copy()  
    results_big = results_compiled.copy()
    #results_compiled.loc['total'] = results_compiled.select_dtypes(pd.np.number).sum()
    display(results_compiled) 
    #save to file
    filename = "%s.xlsx" % variable_of_interest
    results_compiled.to_excel(filename)
    

EPRD_DOM


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,EPRD_DOM2014,EPRD_DOM2015,EPRD_DOM2016,EPRD_DOM2017,EPRD_DOM2018,EPRD_DOM2019
Aerospace & Defence,6197.2507,6758.6557,7575.0992,10354.7092,10158.7292,11134.4426
Agri-food,2907555.0551,3042995.2080,3089239.3275,3133642.0599,3069168.8242,3132585.9461
Construction,31645.7526,32540.3564,35170.7957,50125.5964,49787.8731,52260.0859
Cultural and Creative Industries,9517.1856,9679.4288,9883.1592,15334.0571,15135.8081,13476.9591
Digital,6359.4132,6699.5224,6951.8033,11512.8314,11291.4228,11523.9313
Electronics,2240.8124,2514.1397,2610.9762,3570.5152,3509.9880,4240.7411
Energy - Renewables,4393358.9249,4494631.0413,4560009.2334,4650524.6855,4599519.4435,4612203.9112
Energy Intensive Industries,29553130.5517,30774957.0142,30884961.1802,31921427.7665,31913345.0960,31597919.5698
Health,25901.1521,26626.4195,29436.0568,35124.0610,35900.0693,40941.3140
Mobility - Transport - Automotive,23109.9006,25394.0858,26944.3139,35136.7599,38523.0877,38740.5087


EPRD_HHCNS


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


...

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,NRG_INP_OUT2014,NRG_INP_OUT2015,NRG_INP_OUT2016,NRG_INP_OUT2017,NRG_INP_OUT2018,NRG_INP_OUT2019
Aerospace & Defence,475619.4551,486717.3782,500185.4602,500964.9119,514716.5009,518854.6546
Agri-food,5628785.7144,5807278.2695,5921627.0303,5976263.7838,5914688.1031,5969425.9468
Construction,1745238.7574,1779544.1863,1851968.1029,1798943.9215,1802539.5375,1824370.4083
Cultural and Creative Industries,403580.6441,419062.1626,419528.7965,415517.2111,405507.4283,391055.4968
Digital,324671.1840,349133.8937,361730.7168,366900.8311,378935.3951,387011.1458
Electronics,137491.7317,139069.3989,142023.6254,141352.0194,141437.6818,137115.3105
Energy - Renewables,8566052.9413,8646560.2779,8627681.3672,8723115.1776,8584180.9720,8398245.7177
Energy Intensive Industries,42634259.3614,44028802.3196,44284677.0869,45918139.6818,45709152.0503,45214379.1634
Health,1130824.2763,1137682.8242,1155304.6305,1209442.1791,1186135.6098,1155367.4235
Mobility - Transport - Automotive,3328974.0899,3482673.2767,3547929.8711,3718591.2098,3829089.7273,3747897.2248


WST_USE


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,WST_USE2014,WST_USE2015,WST_USE2016,WST_USE2017,WST_USE2018,WST_USE2019
Aerospace & Defence,7496.0685,7095.3780,7330.5908,7226.9540,7104.5012,7474.1759
Agri-food,46668.4378,45333.1512,45592.8157,45850.6044,44607.6848,47516.9715
Construction,37947.5652,35726.5165,36971.9913,36395.0239,35699.8277,37463.4044
Cultural and Creative Industries,6584.8392,6200.7710,6221.0887,6268.8622,6127.5536,6562.4595
Digital,7752.4947,7303.5564,7542.2756,7472.5294,7311.2330,7685.5025
Electronics,2749.3873,2631.0672,2718.6622,2678.4686,2690.9008,2840.8384
Energy - Renewables,177949.4154,186686.9900,191754.9489,188214.0788,184713.8901,187463.7951
Energy Intensive Industries,744674.8066,764393.3820,787510.4065,827484.4551,849392.3805,865899.3826
Health,29077.7116,27955.0931,28532.8384,28972.7602,28419.3403,29496.7814
Mobility - Transport - Automotive,26917.1923,25372.8956,26177.5494,25835.8048,25391.5636,26695.1798
